In [3]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]


In [4]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/home/user/PythonEnvs/llm-zoomcamp-workshop-Lqlt2HeX/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [5]:
answer_llm = df.iloc[0].answer_llm
embedding_model.encode(answer_llm)

array([-4.22446698e-01, -2.24855885e-01, -3.24058533e-01, -2.84758598e-01,
        7.25700706e-03,  1.01186581e-01,  1.03716530e-01, -1.89983383e-01,
       -2.80597191e-02,  2.71588951e-01, -1.15337208e-01,  1.14666179e-01,
       -8.49588662e-02,  3.32365096e-01,  5.52725829e-02, -2.22195774e-01,
       -1.42540991e-01,  1.02519408e-01, -1.52333871e-01, -2.02912673e-01,
        1.98425166e-02,  8.38148594e-02, -5.68632066e-01,  2.32843533e-02,
       -1.67292729e-01, -2.39256829e-01, -8.05462748e-02,  2.57083420e-02,
       -8.15461725e-02, -7.39289150e-02, -2.61550218e-01,  1.92575734e-02,
        3.22908908e-01,  1.90357193e-01, -9.34726340e-05, -2.13165849e-01,
        2.88942475e-02, -1.79527700e-02, -5.92760742e-02,  1.99918225e-01,
       -4.75173257e-02,  1.71633750e-01, -2.45915409e-02, -9.38058943e-02,
       -3.57002765e-01,  1.33263662e-01,  1.94045931e-01, -1.18530847e-01,
        4.56915408e-01,  1.47728130e-01,  3.35945189e-01, -1.86959580e-01,
        2.45954603e-01, -

In [9]:
evaluations = []

for i, row in df.iterrows():
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']

    answer_llm_encode = embedding_model.encode(answer_llm)
    answer_orig_encode = embedding_model.encode(answer_orig)

    evaluations.append(answer_llm_encode.dot(answer_orig_encode))

In [11]:
import numpy as np

np.percentile(evaluations, 75)

31.674304962158203

In [27]:
evaluations = np.array(evaluations)
norm = np.sqrt((evaluations * evaluations).sum())
evaluations_normalized = evaluations / norm

In [33]:
np.percentile(evaluations_normalized, 75)

0.06479034572839737

In [45]:
from rouge import Rouge
rouge_scorer = Rouge()

index = 10
r = df.iloc[index]

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores


{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [46]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']

# Calculate the average of ROUGE-1, ROUGE-2, and ROUGE-L
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

print(rouge_avg)

0.35490034990035496


In [44]:
rouge_scorer = Rouge()

# Lists to store the scores
rouge_1_list = []
rouge_2_list = []
rouge_l_list = []
rouge_avg_list = []

# Compute ROUGE scores for each record
for index, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
    
    rouge_1_list.append(rouge_1)
    rouge_2_list.append(rouge_2)
    rouge_l_list.append(rouge_l)
    rouge_avg_list.append(rouge_avg)

# Add the scores to the DataFrame
df['rouge-1'] = rouge_1_list
df['rouge-2'] = rouge_2_list
df['rouge-l'] = rouge_l_list
df['rouge-avg'] = rouge_avg_list

# Calculate the average ROUGE-2 F1 score across all records
average_rouge_2 = df['rouge-2'].mean()
print(average_rouge_2)

0.20696501983423318
